In [1]:
import autosklearn.classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics

In [2]:
dataset = pd.read_csv('../Datasets/Heart Disease/heart.csv')

In [3]:
X, y = dataset.iloc[:, :-1], dataset.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# TRAIN MODEL

In [8]:
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=45)
automl.fit(X_train.values, y_train)
y_hat = automl.predict(X_test.values)

In [9]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.9922178988326849

In [10]:
import pickle
with open('./models/heart_disease_automl2.pkl', 'wb') as f:
    pickle.dump(automl, f)

# LOAD MODEL

In [4]:
import pickle
with open('./models/heart_disease_automl2.pkl', 'rb') as f:
    automl = pickle.load(f)

len(automl.show_models())

12

# LIME

In [5]:
import lime
import lime.lime_tabular
import tqdm 

In [6]:
continuous_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
categorical_features = X_train.columns.drop(continuous_features).tolist()
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns.tolist(), class_names=['No Disease', 'Disease'], categorical_features=categorical_features, discretize_continuous=True)

In [7]:
text_x = X_test.values

In [8]:
exp_fn = lambda i: explainer.explain_instance(X_test.iloc[i], automl.predict_proba, num_features=len(X_test.columns))
def exp_fn_blk(xtest, exp_fn):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = exp_fn(i)
        exp1.append(exp.as_map()[exp.available_labels()[0]])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x, exp_fn))

In [9]:
import metrics

In [10]:
exp1 = exp_fn_wrap(text_x)
exp2 = exp_fn_wrap(text_x)

  0%|          | 0/257 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [25]:
np.save('./explanations/heart_disease1.npy', exp1)
np.save('./explanations/heart_disease2.npy', exp2)

In [26]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i,j,0])] = exp[i,j,1]
    return enc_exp

In [27]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(X_test.columns))
sb = metrics.calc_stability(enc1, y_test)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [28]:
i, s, sb

((100.0, 0, 257), (0, 257, 66049, 0.0), (66, 257))

In [29]:
X_test_norm = metrics.normalize_test(X_train, X_test)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [30]:
sim

0.29256260533863654

In [31]:
trust = metrics.calc_trust_score(automl, text_x, exp1, 3, X_train.columns.tolist())

100%|██████████| 257/257 [1:01:44<00:00, 14.41s/it]


In [32]:
trust

0.437094682230869

In [33]:
list_monotonicity = []
list_non_sensitivity = []
list_effective_complexity = []

for i in tqdm.tqdm(range(len(X_test))):
    atr = exp1[i]
    sorted_atr = [j for i,j in atr]
    sorted_feat = [i for i,j in atr]
    y = np.zeros(2, dtype=int)
    np.put(y, y_test.iloc[i], 1)
    example = metrics.FeatureAttribution(automl, X_test.to_numpy()[i], y, sorted_atr)
    list_monotonicity.append(example.monotonicity())
    list_non_sensitivity.append(example.non_sensitivity())
    list_effective_complexity.append(example.effective_complexity(sorted_feat, 0.1))

  0%|          | 0/257 [00:00<?, ?it/s]

100%|██████████| 257/257 [21:05<00:00,  4.92s/it]


In [34]:
print(np.mean(list_monotonicity))
print(np.mean(list_non_sensitivity))
print(np.mean(list_effective_complexity))

print(np.median(list_monotonicity))
print(np.median(list_non_sensitivity))
print(np.median(list_effective_complexity))

-0.004596570744430668
0.0
1.821011673151751
-0.021978021978021976
0.0
0.0


# LIME Global

In [7]:
from lime import submodular_pick
import time

start_time = time.time()
exp1 = submodular_pick.SubmodularPick(explainer, X_test.values, automl.predict_proba, sample_size=200, num_features=len(X_test.columns), num_exps_desired=5)
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
exp2 = submodular_pick.SubmodularPick(explainer, X_test.values, automl.predict_proba, sample_size=200, num_features=len(X_test.columns), num_exps_desired=5)
print("--- %s seconds ---" % (time.time() - start_time))

--- 13291.083112239838 seconds ---
--- 18152.132739782333 seconds ---


In [8]:
import pickle
with open('./explanations/heart_disease_lime_global1.pkl', 'wb') as f:
    pickle.dump(exp1, f)

with open('./explanations/heart_disease_lime_global2.pkl', 'wb') as f:
    pickle.dump(exp2, f)

In [5]:
import pickle
with open('./explanations/heart_disease_lime_global1.pkl', 'rb') as f:
    exp1 = pickle.load(f)

with open('./explanations/heart_disease_lime_global2.pkl', 'rb') as f:
    exp2 = pickle.load(f)

In [6]:
def get_feature_imp(sp_obj):
    W_pick=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.sp_explanations]).fillna(0)
    W_pick['prediction'] = [this.available_labels()[0] for this in sp_obj.sp_explanations]
    W=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.explanations]).fillna(0)
    W['prediction'] = [this.available_labels()[0] for this in sp_obj.explanations]
    np.abs(W.drop("prediction", axis=1)).mean(axis=0).sort_values(ascending=False).head(25).sort_values(ascending=True)
    grped_coeff = W.groupby("prediction").mean()
    grped_coeff = grped_coeff.T
    return grped_coeff[0].values

In [7]:
feat_imp1 = get_feature_imp(exp1)
feat_imp2 = get_feature_imp(exp2)

In [8]:
feat_imp1

array([ 0.06329471,  0.10620919,  0.08554447,  0.06129531, -0.01745359,
        0.05298375,  0.03606276,  0.01228702,  0.00654828,  0.00342238,
        0.00123675, -0.00037584, -0.00059639,  0.08919196, -0.03216177,
        0.01913676, -0.00510993, -0.0073985 , -0.00793242,  0.00037999,
       -0.05393071,  0.04317753, -0.01947467, -0.01210148,  0.00231975,
       -0.00099275, -0.01705473, -0.00849762, -0.00956137,  0.00019452,
       -0.06844988, -0.00719401,  0.00480146,  0.0400006 ,  0.00222889,
       -0.01858862, -0.00241536, -0.00084202, -0.00475555, -0.00048806])

In [9]:
def global_identity(feat_imp1, feat_imp2):
    sum = 0
    for i in range(len(feat_imp1)):
        if(feat_imp1[i] == feat_imp2[i]):
            sum += 1
    return sum/len(feat_imp1)

In [10]:
i = global_identity(feat_imp1, feat_imp2)
i

0.0

In [11]:
def normal_fi(feat_imp):
    return np.abs(feat_imp) / np.sum(np.abs(feat_imp))

In [12]:
normal_feat_imp = normal_fi(feat_imp1 + 1e-9)

In [13]:
#Entropy Ratio
Ser = np.sum(normal_feat_imp*np.log(normal_feat_imp))/np.log(1/len(normal_feat_imp))

# Kullback-Leibler Divergence
Skl = np.sum(normal_feat_imp*np.log(normal_feat_imp/(1/len(normal_feat_imp))))

In [14]:
def calc_gini(pfi):
    sum = 0
    for i in range(len(pfi)):
        sum_curr = 0
        for j in range(len(pfi)):
            sum_curr += np.abs(pfi[i]-pfi[j])
        sum += sum_curr
    
    return sum/(2*len(pfi)**2)*(np.sum(pfi)/len(pfi))

In [15]:
Sg = calc_gini(normal_feat_imp)

In [16]:
Ser, Skl, Sg

(0.822489566487143, 0.6548145910764367, 0.0003829172497483332)

In [17]:
def calc_alpha_fi(normal_pfi, alpha):
    j_inst = 0
    sum = 0
    for i in range(len(normal_pfi)-1, -1, -1):
        sum += normal_pfi[i]
        if sum<=alpha:
            j_inst = i
        else:
            break
    return 1- (j_inst/len(normal_pfi))

In [18]:
calc_alpha_fi(normal_feat_imp, 0.8)

0.925

In [19]:
def get_feature_imp_all(sp_obj):
    W_pick=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.sp_explanations]).fillna(0)
    W_pick['prediction'] = [this.available_labels()[0] for this in sp_obj.sp_explanations]
    W=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.explanations]).fillna(0)
    W['prediction'] = [this.available_labels()[0] for this in sp_obj.explanations]
    np.abs(W.drop("prediction", axis=1)).mean(axis=0).sort_values(ascending=False).head(25).sort_values(ascending=True)
    grped_coeff = W.groupby("prediction").mean()
    grped_coeff = grped_coeff.T
    return grped_coeff

In [20]:
class1_feat_imp, class2_feat_imp = get_feature_imp_all(exp1)[0].values, get_feature_imp_all(exp1)[1].values
normal_class1_fi, normal_class2_fi = normal_fi(class1_feat_imp), normal_fi(class2_feat_imp)

In [21]:
np.linalg.norm(normal_class1_fi - normal_class2_fi, ord=2)

0.21682243154577172

In [22]:
def get_limits_and_names(idxs):
    data = {}
    for idx in idxs:
        
        data[idx] = {}
        col_name = ""
        for col in X_train.columns:
            if str(col) in idx:
                col_name = col
                break
        data[idx]['col_name'] = col_name
        
        split_lt = idx.split("<=")
        if len(split_lt) > 1:
            for i in range(len(split_lt)):
                split_lt[i] = split_lt[i].strip()
                try:
                    split_lt[i] = float(split_lt[i])
                    data[idx]['upper'] = split_lt[i]
                except:
                    pass
            split_lt2 = split_lt[0].split("<")
            if len(split_lt2) > 1:
                for i in range(len(split_lt2)):
                    split_lt2[i] = split_lt2[i].strip()
                    try:
                        split_lt2[i] = float(split_lt2[i])
                        data[idx]['lower'] = split_lt2[i]
                    except:
                        pass
        split_gt = idx.split(">")
        if len(split_gt) > 1:
            for i in range(len(split_gt)):
                split_gt[i] = split_gt[i].strip()
                try:
                    split_gt[i] = float(split_gt[i])
                    data[idx]['lower'] = split_gt[i]
                except:
                    pass
    return data

In [24]:
mean_correctness = 0
feat_scores = get_feature_imp_all(exp1)
feat_scores_index = feat_scores.index.tolist()
exp_feature_name = get_limits_and_names(feat_scores_index)
for i in range(len(X_test)):
    correctness = 0
    for j in range(len(feat_scores_index)):
        inst = 1
        if exp_feature_name[feat_scores_index[j]]['col_name']:
            if 'upper' in exp_feature_name[feat_scores_index[j]].keys():
                if X_test.iloc[i][exp_feature_name[feat_scores_index[j]]['col_name']] > exp_feature_name[feat_scores_index[j]]['upper']:
                    inst = 0
            if 'lower' in exp_feature_name[feat_scores_index[j]].keys():
                if X_test.iloc[i][exp_feature_name[feat_scores_index[j]]['col_name']] <= exp_feature_name[feat_scores_index[j]]['lower']:
                    inst = 0
                    
        # print(inst, feat_scores.iloc[j][y_test.iloc[i]], feat_scores_index[j], exp_feature_name[feat_scores_index[j]]['col_name'], X_test.iloc[i][exp_feature_name[feat_scores_index[j]]['col_name']])
        correctness += inst * 1
    if correctness < 0:
        correctness = 0
    mean_correctness += correctness/len(feat_scores_index)
print(mean_correctness/len(X_test))

0.32500000000000157


# CIU

In [6]:
from ciu import determine_ciu
import tqdm

In [7]:
feat_list = X_train.columns.tolist()

In [8]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = determine_ciu(X_test.iloc[i:i+1], automl.predict_proba, X_train.to_dict('list'), samples = 1000, prediction_index = 1)
        exp_list = [[feat_list.index(i), exp.ci[i]] for i in exp.ci]
        exp1.append(exp_list)
    return np.array(exp1)

In [9]:
exp1 = exp_fn_blk(X_test)
exp2 = exp_fn_blk(X_test)

  0%|          | 0/257 [00:00<?, ?it/s]

  0%|          | 0/257 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [39]:
np.save('./explanations/heart_disease_ciu1.npy', exp1)
np.save('./explanations/heart_disease_ciu2.npy', exp2)

In [10]:
exp1 = np.load('./explanations/heart_disease_ciu1.npy')
exp2 = np.load('./explanations/heart_disease_ciu2.npy')

In [40]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(feat_list))
sb = metrics.calc_stability(enc1, y_test)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [41]:
i, s, sb

((100.0, 0, 257), (0, 257, 66049, 0.0), (46, 257))

In [42]:
X_test_norm = metrics.normalize_test(X_train, X_test)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [43]:
sim

0.5813001331891391

In [44]:
list_monotonicity = []
list_non_sensitivity = []
list_effective_complexity = []

for i in tqdm.tqdm(range(len(X_test))):
    atr = exp1[i]
    sorted_atr = [j for i,j in atr]
    sorted_feat = [i for i,j in atr]
    y = np.zeros(2, dtype=int)
    np.put(y, y_test.iloc[i], 1)
    example = metrics.FeatureAttribution(automl, X_test.to_numpy()[i], y, sorted_atr)
    list_monotonicity.append(example.monotonicity())
    list_non_sensitivity.append(example.non_sensitivity())
    list_effective_complexity.append(example.effective_complexity(sorted_feat, 0.1))

100%|██████████| 257/257 [21:20<00:00,  4.98s/it]


In [45]:
print(np.mean(list_monotonicity))
print(np.mean(list_non_sensitivity))
print(np.mean(list_effective_complexity))

print(np.median(list_monotonicity))
print(np.median(list_non_sensitivity))
print(np.median(list_effective_complexity))

0.12263223158164792
0.0
2.1945525291828796
0.12637362637362637
0.0
0.0


In [13]:
metrics.calc_trust_score(automl, X_test.to_numpy(), exp1, 3, X_train.columns.to_list())

100%|██████████| 257/257 [1:52:15<00:00, 26.21s/it]   


0.4617380025940337

# RULEFIT

In [14]:
from skrules import SkopeRules
import metrics_rules
import time

In [15]:
clf = SkopeRules(max_depth_duplication=2,
                    n_estimators=512,
                    precision_min=0.3,
                    recall_min=0.1,
                    feature_names=X_train.columns.tolist())

In [16]:
start_time = time.time()
clf.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
top_rules1 = clf.score_top_rules(X_test)
top_rules2 = clf.score_top_rules(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.04627823829650879 seconds ---


In [8]:
i = metrics_rules.calc_identity_rules(top_rules1, top_rules2)
print(i)

s = metrics_rules.calc_separability_rules(top_rules1)
print(s)

enc_rules = metrics_rules.exp_enc(clf, top_rules1)
sb = metrics_rules.calc_stability_rules(enc_rules, y_test)
print(sb)

(0.0, 257, 257)
(19630, 257, 66049, 29.72035912731457)
(50, 257)


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [9]:
X_test_norm = metrics_rules.normalize_test(X_train, X_test)
sim = metrics_rules.calc_similarity(enc_rules, X_test_norm)
print(sim)

0.0


# RULEMATRIX

In [5]:
continuous_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
categorical_features = X_train.columns.drop(continuous_features).tolist()

In [6]:
import rulematrix
import time
import metrics_rules

In [7]:
is_continuous = [True if i in continuous_features else False for i in X_train.columns.tolist()]
is_categorical = [True if i in categorical_features else False for i in X_train.columns.tolist()]

In [8]:
surrogate = rulematrix.surrogate.rule_surrogate(
    automl.predict,
    X_train,
    sampling_rate=4,
    is_continuous=is_continuous,
    is_categorical=is_categorical,
    seed=42
)

In [9]:
test_x = X_test.to_numpy()

In [10]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in range(len(xtest)):
        queried_rules = np.arange(surrogate.student.n_rules)[surrogate.student.decision_path(test_x[i].reshape(1,-1)).reshape(-1)]
        exp1.append(queried_rules[-1])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x))

In [11]:
start_time = time.time()
exp1 = exp_fn_blk(test_x)
exp2 = exp_fn_blk(test_x)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.17894291877746582 seconds ---


In [12]:
def enc_exp(exp, n_features):
    enc = []
    for i in range(exp.shape[0]):
        new = np.zeros(n_features)
        for j in surrogate.student.rule_list[exp[i]].clauses:
            new[j.feature_idx] = 1
        enc.append(new)
    return np.array(enc)

In [13]:
enc_exp = enc_exp(exp1, X_train.shape[1])

In [14]:
i = metrics_rules.calc_identity_rules(exp1, exp2)
print(i)

s = metrics_rules.calc_separability_rules(exp1)
print(s)

sb = metrics_rules.calc_stability_rules(enc_exp, y_test)
print(sb)

(0.0, 257, 257)


(5108, 257, 66049, 7.733652288452513)
(53, 257)


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [15]:
X_test_norm = metrics_rules.normalize_test(X_train, X_test)
sim = metrics_rules.calc_similarity(enc_exp, X_test_norm)

In [16]:
sim

1.243738795275592

# ANCHOR Global


In [5]:
from anchor import anchor_tabular
import anchor_utils
import tqdm

In [6]:
explainer = anchor_tabular.AnchorTabularExplainer(
    y_train.unique().tolist(),
    X_train.columns.tolist(),
    X_train.values
)

In [7]:
# Feature Importance using Anchor
def calc_fi(X_test, model, explainer):
    all_exps = []
    for i in tqdm.tqdm(range(len(X_test))):
        exp = explainer.explain_instance(X_test.values[i], model.predict, threshold=0.9)
        all_exps.append(exp.exp_map)
    # fi = anchor_utils.greedy_pick_anchor(all_exps, X_test.values, k = len(X_test.columns))
    return all_exps
        

In [8]:
def get_all_fi(exp):
    fi = []
    for i in exp:
        fi.append(i.exp_map)
    return fi

In [13]:
exp1 = calc_fi(X_test, automl, explainer)
exp2 = calc_fi(X_test, automl, explainer)

100%|██████████| 257/257 [2:04:47<00:00, 29.13s/it]  


In [15]:
X_1 = X_test[y_test == 1]
X_0 = X_test[y_test == 0]

exp_1 = calc_fi(X_1, automl, explainer)
exp_0 = calc_fi(X_0, automl, explainer)

import pickle
with open('./explanations/thyroid_anc_global_class1.pkl', 'wb') as f:
    pickle.dump(exp_1, f)

with open('./explanations/thyroid_anc_global_class0.pkl', 'wb') as f:
    pickle.dump(exp_0, f)

100%|██████████| 125/125 [56:45<00:00, 27.24s/it]


In [14]:
exp_fi1 = anchor_utils.greedy_pick_anchor(exp1, X_test.values, k = len(X_test.columns))
exp_fi2 = anchor_utils.greedy_pick_anchor(exp2, X_test.values, k = len(X_test.columns))

0 0.2607003891050584
1 0.5097276264591439
2 0.5719844357976653
3 0.6303501945525292
4 0.6770428015564203
5 0.6964980544747081
6 0.7159533073929961
7 0.7354085603112841
8 0.754863813229572
9 0.7704280155642024
10 0.7859922178988327
11 0.7976653696498055
12 0.8093385214007782
0 0.2607003891050584
1 0.5097276264591439
2 0.5719844357976653
3 0.6303501945525292
4 0.6770428015564203
5 0.7003891050583657
6 0.7198443579766537
7 0.7392996108949417
8 0.7587548638132295
9 0.7782101167315175
10 0.7937743190661478
11 0.8093385214007782
12 0.8249027237354085


In [16]:
import pickle
with open('./explanations/heart_disease_anc_global1.pkl', 'wb') as f:
    pickle.dump(exp_fi1, f)

with open('./explanations/heart_disease_anc_global2.pkl', 'wb') as f:
    pickle.dump(exp_fi2, f)

In [9]:
import pickle
with open('./explanations/heart_disease_anc_global1.pkl', 'rb') as f:
    exp_fi1 = pickle.load(f)

with open('./explanations/heart_disease_anc_global2.pkl', 'rb') as f:
    exp_fi2 = pickle.load(f)

In [10]:
def normal_fi(feat_imp):
    feat_imp = np.array(feat_imp) + 1e-9
    return np.abs(feat_imp) / np.sum(np.abs(feat_imp))

In [11]:
normal_feat_imp1 = normal_fi(exp_fi1)
normal_feat_imp2 = normal_fi(exp_fi2)

In [12]:
def global_identity(feat_imp1, feat_imp2):
    sum = 0
    for i in range(len(feat_imp1)):
        if(feat_imp1[i] == feat_imp2[i]):
            sum += 1
    return sum/len(feat_imp1)

i = global_identity(normal_feat_imp1, normal_feat_imp2)
i


0.0

In [13]:
#Entropy Ratio
Ser = np.sum(normal_feat_imp1*np.log(normal_feat_imp1))/np.log(1/len(normal_feat_imp1))

# Kullback-Leibler Divergence
Skl = np.sum(normal_feat_imp1*np.log(normal_feat_imp1/(1/len(normal_feat_imp1))))

In [14]:
def calc_gini(pfi):
    sum = 0
    for i in range(len(pfi)):
        sum_curr = 0
        for j in range(len(pfi)):
            sum_curr += np.abs(pfi[i]-pfi[j])
        sum += sum_curr
    
    return sum/(2*len(pfi)**2)*(np.sum(pfi)/len(pfi))

In [15]:
Sg = calc_gini(normal_feat_imp1)

In [16]:
Ser, Skl, Sg

(0.8654685403511111, 0.3450663809847803, 0.002611138049028884)

In [17]:
def calc_alpha_fi(normal_pfi, alpha):
    j_inst = 0
    sum = 0
    for i in range(len(normal_pfi)-1, -1, -1):
        sum += normal_pfi[i]
        if sum<=alpha:
            j_inst = i
        else:
            break
    return 1- (j_inst/len(normal_pfi))

In [18]:
calc_alpha_fi(normal_feat_imp1, 0.8)

0.9230769230769231

In [19]:
import pickle
with open('./explanations/thyroid_anc_global_class1.pkl', 'rb') as f:
    exp_1 = pickle.load(f)

with open('./explanations/thyroid_anc_global_class0.pkl', 'rb') as f:
    exp_0 = pickle.load(f)

In [20]:
np.linalg.norm(normal_fi(exp_0) - normal_fi(exp_1), ord=2)

0.24735085732078507